In [2]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

import sys

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'

firefox = webdriver.Firefox(capabilities=firefox_capabilities, executable_path=geckoPath)
driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')

try:
    firefox.get('https://www.crunchbase.com/organization/apple')
except:
    pass
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(2)
driver.set_page_load_timeout(30)

def existence_in_crunchbase(name):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+name
    url_crunchbase = None
    try:
        driver.get(url)
    except:
        print('timeout of phantomjs')
        pass
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url not in url_crunchbase:
            url_crunchbase = None
    
    return url_crunchbase
            

def extract_company_techcrunch(name,url):
    try:
        firefox.get(url)
    except:
        pass
    founded = None
    employees = None
    company_name = ''
    blocks_dt = []
    blocks_dd = []
    try:
        soup = BeautifulSoup(firefox.page_source, "html.parser")
        blocks_dt = soup.select("div.details dt")
        blocks_dd = soup.select("div.details dd")
        company_name_tag = soup.select_one("#profile_header_heading")
        if company_name_tag == None:
            company_name = ''
        else:
            company_name = company_name_tag.getText()
    except:
        pass

    i = 0
    while company_name.lower() != name.lower() and len(blocks_dt)!=0 :
        time.sleep(0.3)
        try:
            soup = BeautifulSoup(firefox.page_source, "html.parser")
            company_name_tag = soup.select_one("#profile_header_heading")
            if company_name_tag:
                company_name = company_name_tag.getText()
            else:
                company_name = ''
            blocks_dt = soup.select("div.details dt")
            blocks_dd = soup.select("div.details dd")
        except KeyboardInterrupt:
            sys.exit()
        except:
            pass
        i=i+1
        if i >= 100:
            print('fail to crawl ', name , ' in crunchbase')
            break

    
    if company_name.lower() == name.lower():
        for index,block in enumerate(blocks_dt):
            if 'Founded' in  block.getText() : 
                founded = blocks_dd[index].getText()
            if 'Employees' in block.getText():
                employees = blocks_dd[index].getText().split('|')[0]
    print( name , founded , employees)
    return founded,employees



In [3]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
            if "search_label" not in company:
                url_crunchbase = existence_in_crunchbase(name)
                
                if url_crunchbase != None:
                    print(name, ' in crunchbase')
                    i=i+1
                    founded,employees = extract_company_techcrunch(name,url_crunchbase)
                    company["search_label"] = url_crunchbase
                    if founded is not None:
                        company["foundationDate"] = founded
                    if employees is not None:
                        company["number_of_employees"] = employees
                    
                else:
                    company["search_label"] = str(0)
                    print(name, 'not in crunchbase')
            else:
                print("skip: ", name)
            
            if (index+1)%20 == 0:
                self.save_to_disk()
        self.save_to_disk()            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [4]:
CM = CompaniesManager()
CM.extract_companies()
try:
    CM.extend_crunch()
except KeyboardInterrupt:
    sys.exit()

SENSORO  in crunchbase
SENSORO July, 2013 51 - 100 
Microsoft Corp not in crunchbase
Sumitomo not in crunchbase
Manchester City Verve not in crunchbase
SalesWings  in crunchbase
SalesWings None None
SA tech not in crunchbase
Avito not in crunchbase
timeout of phantomjs


SystemExit: 

c:\users\xizhou\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
